In [1]:
import pandas as pd
import numpy as np

In [2]:
import torch

In [3]:
torch.__version__

'1.2.0'

In [4]:
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [14]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set.head()

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561


In [15]:
len(training_set)

79999

In [16]:
training_set = training_set.values
training_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]], dtype=int64)

In [17]:
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set

,1,6,5,887431973
0,1,10,3,875693118
1,1,12,5,878542960
2,1,14,5,874965706
3,1,17,3,875073198
4,1,20,4,887431883
...,...,...,...,...
19994,458,648,4,886395899
19995,458,1101,4,886397931
19996,459,934,3,879563639
19997,460,10,3,882912371


In [18]:
test_set = test_set.values
test_set

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]], dtype=int64)

In [19]:
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_users

943

In [20]:
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))
nb_movies

1682

In [21]:
#creating the data structure 
def convert(data):
    res = []
    for i in range(nb_users):
        movies = np.zeros(nb_movies)
        for j in range(len(data)):
            if data[j, 0] == i+1:
                movies[data[j, 1] - 1] = data[j, 2]
        res.append(movies)
    return np.array(res)

In [22]:
training_set = convert(training_set)

In [23]:
len(training_set), len(training_set[0])

(943, 1682)

In [24]:
training_set[0]

array([0., 3., 4., ..., 0., 0., 0.])

In [25]:
test_set = convert(test_set)
test_set

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [26]:
len(test_set), len(test_set[0])

(943, 1682)

In [27]:
if 5 in test_set[942]:
    print("yes")
else:
    print("no")

no


In [28]:
#converting into torch tensors
training_set = torch.FloatTensor(training_set)
training_set

tensor([[0., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

In [29]:
type(training_set)

torch.Tensor

In [30]:
test_set = torch.FloatTensor(test_set)
test_set

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [31]:
training_set[training_set == 0] = -1

In [32]:
training_set[(training_set == 1) | (training_set == 2)] = 0

In [33]:
training_set[(training_set == 3) | (training_set == 4) | (training_set == 5)] = 1

In [34]:
training_set

tensor([[-1.,  1.,  1.,  ..., -1., -1., -1.],
        [ 1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [ 1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1.,  1., -1.,  ..., -1., -1., -1.]])

In [35]:
test_set[test_set == 0] = -1
test_set[(test_set == 1) | (test_set == 2)] = 0
test_set[test_set >= 3] = 1

In [36]:
test_set

tensor([[-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.]])

In [37]:
#building the architecture of RBM
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

In [38]:
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh) #creating the object of the above coded class

In [39]:
#training our RBM
epochs = 10
for epoch in range(1, epochs + 1):
    train_loss, counter = 0, 0.0
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user: id_user + batch_size]
        v0 = training_set[id_user: id_user + batch_size]
        ph0, _ = rbm.sample_h(v0)
        for k in range(10):
            _, hk = rbm.sample_h(vk)
            _, vk = rbm.sample_v(hk)
            vk[v0 < 0] = v0[v0 < 0]
        phk, _ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
        counter += 1
    print("epoch : " + str(epoch) + " loss: " + str(train_loss/counter))       

epoch : 1 loss: tensor(0.3355)
epoch : 2 loss: tensor(0.2484)
epoch : 3 loss: tensor(0.2582)
epoch : 4 loss: tensor(0.2394)
epoch : 5 loss: tensor(0.2505)
epoch : 6 loss: tensor(0.2452)
epoch : 7 loss: tensor(0.2499)
epoch : 8 loss: tensor(0.2489)
epoch : 9 loss: tensor(0.2460)
epoch : 10 loss: tensor(0.2464)


In [41]:
#testing
test_loss, counter = 0, 0.0
for id_user in range(nb_users):
    v = training_set[id_user: id_user + 1]
    vt = test_set[id_user: id_user + 1]
    if len(vt[vt >= 0]) > 0:
        _, h = rbm.sample_h(v)
        _, v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt >= 0] - v[vt >= 0]))
        counter += 1
print("test loss: " + str(test_loss/counter))       

test loss: tensor(0.2460)


In [44]:
rbm.W

tensor([[ 1.3554, -1.5456, -1.1368,  ...,  4.3065,  4.7412,  3.0129],
        [ 5.8440, -1.1875,  4.2146,  ...,  2.1079, -0.5320,  0.4637],
        [ 8.2193,  3.0058, -0.5755,  ...,  1.1005,  1.5094,  0.2287],
        ...,
        [-0.4770,  2.4359,  1.5226,  ...,  1.9283,  1.0619,  1.8226],
        [ 1.2151, -2.7608,  0.1303,  ...,  3.1398,  4.4964,  1.7334],
        [-0.0849,  2.2478,  5.6506,  ...,  6.6212,  4.8106,  8.2263]])

In [49]:
rbm.W

tensor([[ 1.3554, -1.5456, -1.1368,  ...,  4.3065,  4.7412,  3.0129],
        [ 5.8440, -1.1875,  4.2146,  ...,  2.1079, -0.5320,  0.4637],
        [ 8.2193,  3.0058, -0.5755,  ...,  1.1005,  1.5094,  0.2287],
        ...,
        [-0.4770,  2.4359,  1.5226,  ...,  1.9283,  1.0619,  1.8226],
        [ 1.2151, -2.7608,  0.1303,  ...,  3.1398,  4.4964,  1.7334],
        [-0.0849,  2.2478,  5.6506,  ...,  6.6212,  4.8106,  8.2263]])